# Ce notebook permet de préparer la table et les variables pour réaliser le score de churn du TP

# Sommaire

1. [Import](#sect1)
2. [Nettoyage des données](#sect2)
3. [Export des données pour le notebook "ModelesPridictifChurn"](#sect3)
4. [Analyse descriptive sur les données normalisées](#sect4)

In [ ]:
import pandas as pd
import numpy as np
import os
repertoire = "../data/"
os.chdir(repertoire)

# 1. Import <a name="sect1" ></a> 

### Data -> données de score d'attrition dans le secteur des Telco

In [ ]:
churn_df = pd.read_csv('churn.csv')

In [ ]:
churn_df.shape

In [ ]:
churn_df.head()

In [ ]:
churn_df.columns

In [ ]:
churn_df.dtypes


# 2. Nettoyage des données <a name="sect2" ></a> 

In [ ]:
# Types des colonnes pour transformation
feat_quali = ['Area Code']
feat_quanti = [ 'Account Length', 'VMail Message', 'Day Mins', 'Day Calls', 'Day Charge',
       'Eve Mins', 'Eve Calls', 'Eve Charge', 'Night Mins', 'Night Calls',
       'Night Charge', 'Intl Mins', 'Intl Calls', 'Intl Charge',
       'CustServ Calls']
feat_bool = ["Int'l Plan",'VMail Plan']
target = "Churn?"

In [ ]:
# Area Code -> Catégorielle
churn_df['Area Code']=churn_df['Area Code'].astype(str)


In [ ]:
# Transformation de la variable Target en numérique
churn_df.loc[churn_df['Churn?'] == "True.",'Churn?'] = 1
churn_df.loc[churn_df['Churn?'] == "False.",'Churn?'] = 0

y = churn_df['Churn?'].astype(int)
# 14,5% de Churners
print(y.mean())

In [ ]:
# Transformation des booléens en Numpy Bool
for col in feat_bool:
        churn_df[col] = churn_df[col] == "yes"

In [ ]:
stats = churn_df.describe(include='all')
stats.T

##### Analyse de quelques variables catégorielles 

In [ ]:
# Analyse des liens entre la variable catégorielle "Area Code" afin de recoder : 
# semble ne pas avoir de lien mais conservation de la colonne pour des liens non binaires
print(pd.crosstab(churn_df['Area Code'],y, normalize='index'))


In [ ]:
# par contre les deux var qui indiquent si le client paie pour un service international ou VM sont très discriminantes
print(pd.crosstab(churn_df["Int'l Plan"],y, normalize='index'))
print(pd.crosstab(churn_df["VMail Plan"],y, normalize='index'))

In [ ]:
# étude du state
print(pd.crosstab(churn_df['State'],y,normalize='index'))

In [ ]:
len(churn_df['State'].unique())
# 51 valeurs
# soit on garde et on crée des dummy
# soit on créé des regroupements "métiers" (non abordé dans la formation)

Dichotomisation des variables qualitative 

In [ ]:
# les dummies
churn_df = pd.get_dummies(churn_df,columns=feat_quali, drop_first=True)

In [ ]:
churn_df.head()

In [ ]:
#############################################################################################
# Traitement du state en  remplacant l'état par la moyenne de y dans chaque état
# Faire ce travail c'est déjà faire un modèle simple donc il faut absolument refaire l'échantillon d'apprentissage
#############################################################################################
# split Apprentissage Test
from sklearn.model_selection import train_test_split 
state_train, state_test, y_train, y_test = train_test_split(churn_df['State'],y,  test_size=0.3,random_state=42)


In [ ]:
taux_reponse_state_train=pd.crosstab(state_train,y_train).apply(lambda r: r/r.sum(), axis=1)[1]
taux_reponse_state_test=pd.crosstab(state_test,y_test).apply(lambda r: r/r.sum(), axis=1)[1]
print(taux_reponse_state_train) # churn par état

In [ ]:
print(taux_reponse_state_test) 

In [ ]:
del state_test,y_test,  taux_reponse_state_test
del state_train
# on remplace le state par le taux de reponse associé
list(taux_reponse_state_train)
# replace une liste par une liste
churn_df['State'].replace(list(taux_reponse_state_train.index), list(taux_reponse_state_train), inplace=True)
churn_df.rename(columns={"State": "churnMoy_state"},inplace=True)

Suppression de variables inutiles

In [ ]:
to_drop=['Phone']
churn_df = churn_df.drop(to_drop,axis=1)
churn_df.describe()


In [ ]:
churn_df.columns

# 3. Export <a name="sect3" ></a> 

In [ ]:
import pickle
file=open("churn_prepared.pydata","wb")
pickle.dump(churn_df,file)
file.close()

# 4. Analyse descriptive <a name="sect4" ></a>

In [ ]:
# Décupage entre variables explicatives et target
X = churn_df.drop(target, axis=1)
y = churn_df[target]

D'abord on normalise les données afin d'avoir des analyses comparables sur la même échelle dans les box plot ci-dessous

In [ ]:
from sklearn.preprocessing import StandardScaler
norm=StandardScaler().fit(X) 
X_norm = norm.transform(X) 
# ou 
# X = StandardScaler().fit_transform(X)

In [ ]:
X_norm = pd.DataFrame(X_norm)
# on récupère les noms de col
X_norm.columns = X.columns
X_norm.shape


In [ ]:
# Taux moyen de 14.4% de churners
y.mean()

In [ ]:
X_norm.describe()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Calcul du nb de lignes et colonnes à ajouter pour la data viz
nb_feat = len(X_norm.columns) 
n_col = 3
n_rows = nb_feat // n_col
if (nb_feat % n_col) > 0:
    n_rows += 1

n_col,  n_rows

In [ ]:
fig, axes = plt.subplots(n_rows, n_col, figsize=(15, 30), sharey=True)

r = 0
c = 0
for col in X_norm.columns:
    if c < (n_col - 1):
        sns.boxplot(ax=axes[r][c], x = y, y=X_norm[col])
        
    elif c == (n_col - 1):
        sns.boxplot(ax=axes[r][c], x = y, y=X_norm[col])
        r += 1
    else:
        c = 0
        sns.boxplot(ax=axes[r][c], x = y, y=X_norm[col])
    c += 1

In [ ]:
# tri croisé des var quali
for i in ("Int'l Plan","VMail Plan","Area Code_415","Area Code_510"):
    print(pd.crosstab(X_norm[i], y, normalize=0))